In [3]:
import json
import pandas as pd
from datasets import load_dataset

ds = load_dataset("MatrixStudio/Codeforces-Python-Submissions", split="train")

print(ds)

/opt/conda/envs/rllm-xiao/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['contestId', 'index', 'name', 'type', 'rating', 'tags', 'title', 'time-limit', 'memory-limit', 'problem-description', 'input-specification', 'output-specification', 'demo-input', 'demo-output', 'note', 'points', 'test_cases', 'creationTimeSeconds', 'relativeTimeSeconds', 'programmingLanguage', 'verdict', 'testset', 'passedTestCount', 'timeConsumedMillis', 'memoryConsumedBytes', 'code', 'prompt', 'response', 'score'],
    num_rows: 621356
})


In [4]:
# func. to extract difficulty rating from tags
def extract_difficulty(tags):
    if pd.isnull(tags):  # Handles when tags are null
        return 0
    for tag in tags.split(","):
        if "*" in tag:  # Difficulty is marked with a '*' symbol
            try:
                return int(tag.replace("*", ""))
            except ValueError:
                continue
    return 0  # Default difficulty 

In [5]:
dataset = []
for entry in ds:
    new_entry = {
        "problem": entry["prompt"] + "\n" + entry["problem-description"],#Note(Xiao):this may have problem
        "test_cases": entry["test_cases"],#str, when use it as a reward, need use json.loads
    }
    dataset.append(new_entry)

print(f"train dataset size: {len(dataset)}")

with open("train_codeforces.json", "w") as f:
    json.dump(dataset, f, indent=4)

train dataset size: 621356


In [7]:
ds = load_dataset("Qwen/CodeElo", split="test")

print(ds)

Generating test split: 100%|██████████| 408/408 [00:00<00:00, 1875.96 examples/s]

Dataset({
    features: ['problem_id', 'url', 'title', 'rating', 'tags', 'div', 'time_limit_ms', 'memory_limit_mb', 'description', 'input', 'output', 'interaction', 'examples', 'note'],
    num_rows: 408
})


In [12]:
TEMPLATE = """
{description}

Input Format:
{input_format}

Output Format:
{output_format}

Tags:
{tags}

Time Limit: {time_limit} ms
Memory Limit: {memory_limit} MB
"""

In [13]:
dataset = []
for entry in ds:
    new_entry = {
        "problem": TEMPLATE.format(
            description=entry["description"],
            input_format=entry["input"],
            output_format=entry["output"],
            tags=entry["tags"],
            time_limit=entry["time_limit_ms"],
            memory_limit=entry["memory_limit_mb"],
        ),
        "test_cases": entry["examples"],#str, when use it as a reward, need use json.loads
    }
    dataset.append(new_entry)

print(f"test dataset size: {len(dataset)}")

with open("test_codeforces.json", "w") as f:
    json.dump(dataset, f, indent=4)

test dataset size: 408
